In [ ]:
!pip install ipython

In [ ]:
# Python Playground

import json
import os
import re
from pathlib import Path
from pprint import pp

import IPython
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import requests
from IPython.display import HTML, JSON, Code, Image, Markdown, display
from langchain_community.document_loaders import JSONLoader

%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
home_dir = os.path.expanduser("~")
npc_path = os.path.join(home_dir, 'Desktop/tmp/npc.json')
topics_path = os.path.join(home_dir, 'Desktop/tmp/topics.json')

In [ ]:
if (windows) {
  function groupPairs(arr) {
    let result = [];
    for (let i = 0; i < arr.length; i += 2) {
        result.push(arr.slice(i, i + 2));
    }
    return result;
  }

  var topic_consumers_list = [...document.querySelectorAll("#widget_1671671868820084 > div > div > div.widget-overlay.widget.query_table.widget--with_title.multisize_layout > div > div:nth-child(3) > div > div > div > div > div.druids_table_table__root-wrapper > div.druids_table_table__root > div > table > tbody > tr > td:first-child, #widget_1671671868820084 > div > div > div.widget-overlay.widget.query_table.widget--with_title.multisize_layout > div > div:nth-child(3) > div > div > div > div > div.druids_table_table__root-wrapper > div.druids_table_table__root > div > table > tbody > tr > td:nth-child(2)")].map(x => x.innerText)

  var npc_topic_n_consumers_pairs = groupPairs(topic_consumers_list)
}

In [ ]:
# 1. Read JSON files into variables
npc_topic_n_consumers = json.loads(Path(npc_path).read_text())
npc = []
consumer = []
for [ npc_x, consumer_x ] in npc_topic_n_consumers:
    npc.append(npc_x)
    consumer.append(consumer_x)
    

JSON(npc)


topics = requests.get("http://kube-kafkarestproxy-c14.service.intradsm1.sdeconsul.csnzoo.com/topics").json()
topics = list(map(lambda x: x.lower(), topics))
JSON(topics)

E2E_REGEX = r'(?i)^e2e(-|_| )'

topics_e2e = [ re.sub(E2E_REGEX, '', x) for x in topics if isinstance(x, str) and re.match(E2E_REGEX, x)]
JSON(topics_e2e)

In [ ]:
# 2. Convert each string in list to reverse alphabetical order
def reverse_alpha_order(s):
    return s[::-1]
npc_reversed = [ reverse_alpha_order(x) for x in npc]
topics_e2e_reversed = list(map(reverse_alpha_order, topics_e2e))

topics_e2e_reversed[0:5]


In [5]:
# 3. Match entries and categorize
# found = list([ {"haystack": reverse_alpha_order(x), "match": reverse_alpha_order(topic_reverse) } for x in npc_reversed  for topic_reverse in topics_e2e_reversed if x in topic_reverse])
# not_found = set([reverse_alpha_order(x) for x in npc_reversed for topic_reverse in topics_e2e_reversed if x in topic_reverse])
found = []
not_found = []

for x in npc_reversed:
    match_found = False
    match_topic = None
    for topic_reverse in topics_e2e_reversed:
        if x == topic_reverse:
            match_found = True
            match_topic = topic_reverse
            break
    if match_found:
        found.append({"haystack": reverse_alpha_order(x), "match": reverse_alpha_order(match_topic)})
    else:
        not_found.append(x)
        


In [ ]:
pp(not_found)

In [ ]:
# Get All Info from Kafka Clusters
kafka_cluser_info = requests.get("http://kube-kafkarestproxy-c14.service.intradsm1.sdeconsul.csnzoo.com/v3/clusters").json()
kafka_cluser_id = kafka_cluser_info["data"][0]["cluster_id"]
kafka_consumer_groups = kafka_cluser_info["data"][0]["consumer_groups"]["related"]
kafka_topics = kafka_cluser_info["data"][0]["topics"]["related"]

kafka_consumer_groups

In [ ]:
consumer_groups_response = requests.get(kafka_consumer_groups).json()

consumer_groups_response

consumer_group_mapping = {consumer['consumer_group_id']: consumer['consumers']['related'] for consumer in consumer_groups_response['data']}

consumer_group_mapping_e2e = {consumer['consumer_group_id']: consumer['consumers']['related'] for consumer in consumer_groups_response['data'] if re.match(E2E_REGEX, consumer['consumer_group_id']) }

len(consumer_group_mapping_e2e), consumer_group_mapping_e2e


In [ ]:
# find all the consumer groups not defined with e2e-prefix
import IPython.display


defined_e2e_consumers = consumer_group_mapping_e2e.keys()
defined_e2e_consumers_without_prefix = [ re.sub(E2E_REGEX, '', x).lower() for x in defined_e2e_consumers ]

consumer

consumer_set = set(consumer)
defined_e2e_consumers_without_prefix_set = set(defined_e2e_consumers_without_prefix)


consumers_not_defined = consumer_set - defined_e2e_consumers_without_prefix_set

len(consumers_not_defined), len(consumer_set)

display(consumers_not_defined)
# consumer_group_mapping

In [ ]:
# get original consumer group names not defined
consumer_group_mapping

consumer_groups_details_not_defined = { k: v for k, v in consumer_group_mapping.items() if k.lower() in consumers_not_defined}

len(consumer_groups_details_not_defined)


In [ ]:
import yaml
import json
from functools import reduce
from typing import Any, Dict, List, Optional

import operator

yaml_string = """
part_deleted:
  topic: "e2e-Wayfair.Supplier.NewProductCreation.PartDeleted"
  groups:
    - name: "ShippingPartDeletedConsumer"
      value: "e2e-ShippingPartDeletedConsumer"
    - name: "CompliancePartDeletedConsumer"
      value: "e2e-CompliancePartDeletedConsumer"
    - name: "FeaturesPartDeletedConsumer"
      value: "e2e-FeaturesPartDeletedConsumer"
    - name: "CoreProductPartDeletedConsumer"
      value: "e2e-CoreProductPartDeletedConsumer"
    - name: "CatmanPricingPartDeletedConsumer"
      value: "e2e-CatmanPricingPartDeletedConsumer"
part_submission_status:
  topic: "e2e-NpcSubmissionStatusOutput"
  groups:
    - name: "CoreProductSubmissionStatusConsumer"
      value: "e2e-CoreProductSubmissionStatusConsumer"
part_parsed:
  topic: "e2e-Wayfair.Supplier.NewProductCreation.PartParsed"
  groups:
    - name: "PartParsedConsumer"
      value: "e2e-PartParsedConsumer"
staging_part_save_result:
  topic: "e2e-Wayfair.Supplier.NewProductCreation.Staging.PartSaveResult"
staging_part_save_dead_letter:
  topic: "e2e-Wayfair.Supplier.NewProductCreation.Staging.PartSave.DeadLetter"
staging_part_save_retry:
  topic: "e2e-Wayfair.Supplier.NewProductCreation.Staging.PartSave.Retry"
  
#components/kafka/example/constants.py topics
kafka:
  topic: "e2e-topic"
  avro_topic: "e2e-avro_topic"

#components/producers/product_created_log/producer.py  
product_created_log:
  topic: "e2e-Wayfair.Supplier.NewProductCreation.Analytics.ProductCreatedLog"  

"""
data_dict = yaml.safe_load(yaml_string)


def get_kafka_group(
    path: List[str], group_name: str = "", default: str = ""
) -> str:
    group_configs = reduce(operator.getitem, path, data_dict)
    print(f'{json.dumps(group_configs)}')
    if isinstance(group_configs, list):
        for group in group_configs:
            if isinstance(group, dict) and "name" in group:
                if group["name"].lower() == group_name.lower():
                    return group["value"]
    return default

get_kafka_group(
    ["part_deleted", "groups"], "CompliancePartDeletedConsumer"
)